# Fine -Tuning Neural Network Hyperparamters

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense,Dropout
import keras_tuner as kt

In [8]:
def build_model(hp):
    model = keras.Sequential()

    for i in range(hp.Int("num_layers",1, 3)):
        model.add(Dense(units = hp.Int(f"units_{i}",min_value=32,max_value=256,step=32),
                       activation = "relu"))
        model.add(Dropout(rate=hp.Float(f"dropout_{i}",0.1,0.5,step = 0.1)))

        #output layer
        model.add(Dense(1,activation="linear"))

        #tune optimizer
        model.compile(optimizer=hp.Choice("optimizer",["adam","sgd","rmsprop"]),
                     loss="mse",
                     metrics = ["mae"]
                     )
        return model    

In [9]:
tuner = kt.RandomSearch(
    build_model,
    objective = "val_mae",
    max_trials = 10,
    executions_per_trial = 2,
    directory = "hyper_tuning",
    project_name = "neural_net_tuning"
)

In [10]:
import numpy as np

In [12]:
X_train = np.random.rand(1000,10)
y_train = np.random.rand(1000,1)

tuner.search(X_train,y_train,epochs = 20,validation_split=0.2,batch_size=32)

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best Hyperparamters : {best_hps.values}")

Trial 10 Complete [00h 00m 16s]
val_mae: 0.24481824040412903

Best val_mae So Far: 0.24481824040412903
Total elapsed time: 00h 02m 38s
Best Hyperparamters : {'num_layers': 3, 'units_0': 256, 'dropout_0': 0.30000000000000004, 'optimizer': 'rmsprop'}


In [14]:
best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(X_train,y_train,epochs = 50,validation_split=0.2,batch_size = 32)

Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.1952 - mae: 0.3583 - val_loss: 0.0847 - val_mae: 0.2488
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0991 - mae: 0.2686 - val_loss: 0.0857 - val_mae: 0.2483
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0978 - mae: 0.2642 - val_loss: 0.0846 - val_mae: 0.2479
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.1002 - mae: 0.2697 - val_loss: 0.0833 - val_mae: 0.2498
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0933 - mae: 0.2606 - val_loss: 0.0810 - val_mae: 0.2468
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0910 - mae: 0.2563 - val_loss: 0.0822 - val_mae: 0.2464
Epoch 7/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0844 - mae: 0.2482 - val_loss: 0.0809 - val_mae: 0.2453
Epoch 8/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0883 - mae: 0.2534 - val_loss: 0.0811 - val_mae: 0.2475
Epoch 9/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0876 - mae: